# Error Propagation with QExPy

The first step is to import the error module for error propagation. We'll import it as "e":

In [1]:
import qexpy as q


Loading BokehJS ...

Next, we'll declare two measured values, x and y, with uncertainties, and print them out. We use the Measurement object from the qexpy.error package:

In [2]:
#Our two measured values:
x = q.Measurement(10,1)
y = q.Measurement(5,3)

#We can print them out:
print("x =",x)
print("y =",y)
#x.set_correlation(y,0.3)

x = 10 +/- 1
y = 5 +/- 3


We can declare a third object, z, which depends on x and y. The uncertainty in x and y will be correctly propagated to z, so once we have defined z, we can simply print it out with the correct uncertainty:

In [3]:
#We define z
z = x+y
#z can now be printed out
print("z =",z)

z = 15 +/- 3


Note how the uncertainties have been kept to 1 significant figure. In this case, the error in z was obtained by adding the errors in x and y in quadrature. We can change the number of significant figures to confirm that the errors were indeed added in quadrature. We can choose between setting  the number of significant figures based on the uncertainty (more common) or based on the central value.

In [4]:
q.set_sigfigs_error(4) # set sigfigs based on the error
print("z =",z)

z = 15.000 +/- 3.162


Let's compare the error in z to what the errors in x and y are when added manually in quadrature. We need to import the sqrt() function from the math package to apply mathematical functions to numbers:

In [5]:
import math as m
quadrature = m.sqrt(x.std**2+y.std**2)
print(quadrature)

3.1622776601683795


## Math functions
We can propagate the uncertainties through any operator (+,-,\*,/) automatically as we showed above. QExPy also knows how to propagate the uncertainty through common mathematical functions. To use mathematical functions on Measurement objects, we need to call the functions from the QExPy package (as opposed to the math package as we did above for 2 numbers).

In [6]:
#If an object fell a distance of 8.0 +/0.1 m, how long did it take to fall?
y = q.Measurement(8,0.1)
g = 9.81
t = q.sqrt(2*y/g)
print("time = ",t, "seconds")

time =  1.277102 +/- 0.007982 seconds


## Error in correlated quantities
If we have two measurements, x and y, that are correlated, then their correlation factor will impact the uncertainty on a quantity that depends on them:

In [7]:
x = q.Measurement(10,1)
y = q.Measurement(5,3)
z = x+y
print("x and y uncorrelated: z=",z)

#Now set a correlation factor between x and y:
x.set_correlation(y,0.5)
z = x+y
print("x and y positively correlated: z=",z)

#We can also use the covariance factor instead of the correlation factor:
x.set_covariance(y,-1.5)
z = x+y
print("x and y negatively correlated: z=",z)

x and y uncorrelated: z= 15.000 +/- 3.162
x and y positively correlated: z= 15.000 +/- 3.606
x and y negatively correlated: z= 15.000 +/- 2.646


If we don't specify any correlation factors, then all quantities are assumed to be independent. However, a quantity should not be independent from itself, so QExPy knows how to track the correlation in quantities that depend on common quantities. 

In [8]:
x = q.Measurement(10,1)
y = x*x
z = x*x - y #this should be 0 +/- 0, since it's really x^2 - x^2 
print(z)

0.000 +/- 0.000


## Statistical measurements
QExPy can also handle the case when you have repeated measurements of a single quantity and you want to average them together. Suppose that you have measured 5 values of some quantity T. QExPy will automatically assume that those values should be averaged together so that T is given by the mean of the measured values with an uncertainty given by the standard deviation of the values.


In [9]:
T=q.Measurement( [5.6, 4.8, 6.1, 4.9, 5.1 ] )
print(T)

5.3000 +/- 0.5431


T can be used just as any other measurement with uncertainties, and its error will be propagate correctly:

In [10]:
omega = 2*3.14/T
print(omega)

1.1849 +/- 0.1214


If we have measured many values, it can sometimes be useful to visualize those measurements in a histogram. QExPy will automatically create a histogram of the values, showing lines corresponding to the mean and the range covered by one standard deviation

In [11]:
T.show_histogram()

## Multiple measurements (Arrays of Measurements)
QExPy can also handle the case of having multiple measurements, when each measurement has its own uncertainty. For example, if you have three measurements of a single quantity, in order to get an average value, you should weight each measurement by its uncertainty (or rather the weight should be 1 over the square of the uncertainty). This is handled by the MeasurementArray class:

In [12]:
gvals = q.MeasurementArray([(9.8,0.2), (14,3) , (9.9,0.1)], name="g", units='m/s^2') 
print(gvals)
print("The mean is ",gvals.get_mean())
print("The error weighted mean is ",gvals.get_error_weighted_mean())

g_0 = 9.8000 +/- 0.2000 [m/s^2],
g_1 = 14.000 +/- 3.000 [m/s^2],
g_2 = 9.9000 +/- 0.1000 [m/s^2]
The mean is  11.2333333333
The error weighted mean is  9.72815 +/- 0.08870


The MeasurementArray object truly is an array of Measurement objects, so individual measurements can be retrieved (remember that the first element in an array in python has index 0 not 1, so element 1 is the second element):

In [13]:
print(gvals[1])

g_1 = 14.000 +/- 3.000 [m/s^2]


## Error propagation methods

### Derivative method (default)
By default, QExPy propagates the uncertainties using the "derivative" method. That is, for a function, $f(x,y)$, that depends on measured quantities $x\pm\sigma_x$ and $y\pm\sigma_y$, with covariance $\sigma_{xy}$ between the two measured quantities, the uncertainty in $f$ is given by:
$$ \sigma_f = \sqrt{ \left(\frac{\partial f}{\partial x} \sigma_x \right)^2 + \left(\frac{\partial f}{\partial y} \sigma_y \right)^2 + 2 \frac{\partial f}{\partial x} \frac{\partial f}{\partial y}\sigma_{xy} }$$

QExPy evaluates the derivatives exactly when propagating the uncertainties using an algorithm called "automatic differentiation". This is possible as QExPy internally keeps track of the dependency of quantities on each other, and as an added bonus can also be used to evaluate numerical derivatives exactly. Although the derivative method is commonly taught in undergraduate laboratories, it is only valid when the relative uncertainties in the quantities being propagated are small (e.g. less than ~10% relative uncertainty). This method is thus not strongly encouraged, although it has been made the default because it is so prevalent in undergraduate teaching. 


### Min-Max method (not recommended)
This is not the only way to propagate the uncertainty in $f$. For example, the "Min-Max" method, is a method to yield a more conservative (larger) estimate of the uncertainty in $f$ and is often used in introductory courses. The Min-Max method defines the central value and uncertainty in $f$ as:
$$f=\frac{1}{2}(f^{max}+f^{min})$$
$$\sigma_f=\frac{1}{2}(f^{max}-f^{min})$$

where $f^{max}$ ($f^{min}$) is the maximum (minimum) value that $f$ takes when $x$ and $y$ are varied within their uncertainty range. For example, if $f(x,y)=x+y$, then the maximum and minimum of $f$ are easily found:

$$f^{max} = (x+\sigma_x)+(y+\sigma_y)$$
$$f^{min} = (x-\sigma_x)-(y-\sigma_y)$$

The Min-Max method actually requires a numerical approximation to evaluate the values of $f^{max}$ and $f^{min}$ for all but the most simple cases. Hence although it is a good method to introduce the idea of uncertainty propagation, it is not recommended to use this method in any serious calculation (it also does not take correlations into account). 

### Monte Carlo method (recommended!)
Finally, the recommended method to propagate errors is the "Monte-Carlo" method, although it is the hardest to understand. The MC method is based on a statiscal understanding of the measurements. In the QExPy implementation, currently, the main assumptions is that the uncertainty in a quantity is given by a "standard error"; that is, if $x = 10\pm 1$, then we *assume* that this error and uncertainty should be interpreted as: "if we measure $x$ multiple times, we will obtain a set of measurements that are normally distributed with a mean of 10 and a standard deviation of 1". In other words, we assume that $x$ has a 68% chance of being in the range between 9 and 11.


The MC method then uses the assumption that measured quantities are normally distributed and use this to propagate the errors by using Monte Carlo simulation. Suppose that we have measured $x$ and $y$ and wish to determine the central value and uncertainty in $x=x+y$. The Monte Carlo method will generate normally distributed random values for $x$ and $y$ (the random numbers will be correctly correlated if the user has indicated that $x$ and $y$ are correlated), then it will add those random values together, to obtain a set of values for $z$. The mean and standard deviation of the random values for $z$ are taken as the central value and uncertainty in $z$. 

## What QExPy actually does
Although the user appears to choose the method used to propagate the errors, QExPy always uses all three methods behind the scenes and the user only decides which method to print out. This allows QExPy to compare the results behind the scenes, in particular, to inform the users that the uncertainties using a particular method (e.g. derivative) may be inaccurate and to suggest that the user choose a different method. 

## Example
Below, we illustrate an example of using the different methods to propagate the uncertainty in the Coulomb force based on the measurement of two charges, $q_1$ and $q_2$, and the distance between them, $r$. We illustrate how the derivative method does not give the correct answer if the relative uncertainties in the measured quantities are large. 

### 1 % relative uncertainty calculation


In [14]:
#Measurements with 1% relative uncertainties:
relative_factor = 0.01 

#Measured values
q1m=1e-6
q2m=2e-5
rm=0.1

#Convert to Measurement objects
q1 = q.Measurement(q1m,relative_factor*q1m)
q2 = q.Measurement(q2m,relative_factor*q2m)
r = q.Measurement(rm,relative_factor*rm)
#Coulomb's constant:
k = 9e9 

#Define the Force:
F = k*q1*q2/r**2

#Print out the different errors
q.set_error_method("derivative")
print("Derivative method, F = ",F)
q.set_error_method("minmax")
print("Min-Max method, F =", F)
q.set_error_method("mc")
print("Monte Carlo method, F =",F)



Derivative method, F =  18.0000 +/- 0.4409
Min-Max method, F = 18.0144 +/- 0.7202
Monte Carlo method, F = 18.0035 +/- 0.4433


### 10 % relative uncertainty calculation
We see that in this case, the Monte Carlo method returns a different uncertainty than the derivative method, because the derivative method is incorrect when the uncertainties are this large.

In [15]:
#Measurements with 10% relative uncertainties:
relative_factor = 0.1 

#Measured values
q1m=1e-6
q2m=2e-5
rm=0.1

#Convert to Measurement objects
q1 = q.Measurement(q1m,relative_factor*q1m)
q2 = q.Measurement(q2m,relative_factor*q2m)
r = q.Measurement(rm,relative_factor*rm)
#Coulomb's constant:
k = 9e9 

#Define the Force:
F = k*q1*q2/r**2

#Print out the different errors
q.set_error_method("derivative")
print("Derivative method, F = ",F)
q.set_error_method("minmax")
print("Min-Max method, F =", F)
q.set_error_method("mc")
print("Monte Carlo method, F =",F)

q.plot_engine="bokeh"
F.show_MC_histogram()

Derivative method, F =  18.000 +/- 4.409
Min-Max method, F = 19.469 +/- 7.420
Monte Carlo method, F = 18.592 +/- 4.691


## Printing style and formatting
QExPy can print results in different styles, and the user can assign names and units to variables (although one should not rely on units being correctly propagated). If one assigns and name and units to a variable, the print() function will automatically include those when printing out the variable.

One can also set the "print style" to be either "Default", "Scientific" or "Latex". In the scientific and default styles, QExPy will factor out the powers of 10 to make the central value and uncertainty easier to compare. 

In [16]:
x = q.Measurement(10,1.22,name="distance", units="m")
q.set_print_style("Scientific")
print(x)
q.set_print_style("Latex")
print(x)
q.set_print_style("Default")
print(x)

distance = (10000 +/- 1220)*10^(-3) [m]
distance = (10000 \pm 1220)*10^{-3}\,m
distance = 10.000 +/- 1.220 [m]


## Exact derivatives
Since QExPy needs to be able to evaluate derivatives (by using the automatic differentiation algorithm that exploits the Chain Rule), we can use QExPy to evaluate numerical derivatives exactly. That is, given a function, $f(x,y)$, QExPy can evaluate $\frac{\partial f(x,y)}{\partial x}$ at a given value of $x$ and $y$.

For example, if we have:
$$x(t) = \frac{1}{2}gt^2 $$

and we have measured $t=8 \pm 1$, we can evaluate the exact value of the derivative:

$$\frac{\partial x(t)}{\partial t} = \frac{d x(t)}{d t}  = gt$$

at $t=8 \pm 1$


In [17]:
t = q.Measurement(8,1)
x = 0.5 * 9.81 * t**2
print(x.get_derivative(t))

78.48
